<a href="https://colab.research.google.com/github/linneverh/ML/blob/main/API_embedding_test_template.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### Prep datasets

In [2]:
#Library
import numpy as np
import pandas as pd
import sklearn
from sklearn.metrics.pairwise import cosine_similarity
import matplotlib.pyplot as plt
import seaborn as sns
import random
from sklearn.preprocessing import normalize
from sklearn.feature_extraction.text import TfidfVectorizer

In [38]:
# Load the datasets
interactions = pd.read_csv('https://raw.githubusercontent.com/linneverh/MachineLearning/main/interactions_train.csv')

# FOR: author_date_title_description
# items1 = pd.read_csv("https://raw.githubusercontent.com/linneverh/ML/refs/heads/main/author_date_title_description/embeddings_part1.csv")
# items2 = pd.read_csv("https://raw.githubusercontent.com/linneverh/ML/refs/heads/main/author_date_title_description/embeddings_part2.csv")
# items = pd.concat([items1, items2])

#FOR: author_date_title_subjects
# items1 = pd.read_csv("https://raw.githubusercontent.com/linneverh/ML/refs/heads/main/author_date_title_subjects/embeddings_part1.csv")
# items2 = pd.read_csv("https://raw.githubusercontent.com/linneverh/ML/refs/heads/main/author_date_title_subjects/embeddings_part2.csv")
# items = pd.concat([items1, items2])

#FOR: google_api_enhanced/all_items_enhanced - DOESN'T HAVE EMBEDDINGS?
# items1 = pd.read_csv("https://raw.githubusercontent.com/linneverh/ML/refs/heads/main/google_api_enhanced/nan_books_enhanced.csv")
# items2 = pd.read_csv("https://raw.githubusercontent.com/linneverh/ML/refs/heads/main/google_api_enhanced/nan_books_enhanced.csv")
# items = pd.concat([items1, items2])

#FOR: title_description
items1 = pd.read_csv("https://raw.githubusercontent.com/linneverh/ML/refs/heads/main/title_description/embeddings_part1.csv")
items2 = pd.read_csv("https://raw.githubusercontent.com/linneverh/ML/refs/heads/main/title_description/embeddings_part2.csv")
items = pd.concat([items1, items2])

#rename columns
interactions = interactions.rename(columns={'u': 'user_id', 'i': 'book_id', 't': 'timestamp'})
items=items.rename(columns={'i':'book_id'})
# Display the first rows of the updated interactions DataFrame
display(interactions.head())
display(items.head())

# Display the first rows of each dataset
display(interactions.head())
display(items.head())

,user_id,book_id,timestamp
0,4456,8581,1.687541e+09
1,142,1964,1.679585e+09
2,362,3705,1.706872e+09
3,1809,11317,1.673533e+09
4,4384,1323,1.681402e+09


,Unnamed: 0.1,Unnamed: 0,CanonicalLink,Description,ISBN,ImageLink,Language,PublishedDate,Publisher,Subjects,...,google_api_title,book_id,title_clean,title_description,author_title_description,author_date_title_description,author_date_title,author_date_title_subjects,author_title_subjects,embedding
0,0,0,https://play.google.com/store/books/details?id...,NaN,NaN,http://books.google.com/books/content?id=m5qZA...,fr,1932.0,Garnier frères,Philosophy,...,Controverses ; et Suasoires,261,Controverses ; et Suasoires,Controverses ; et Suasoires,"Seneca, Lucius Annaeus, l'Ancien Controverses ...","Seneca, Lucius Annaeus, l'Ancien 1932 Controve...","Seneca, Lucius Annaeus, l'Ancien 1932 Controve...","Seneca, Lucius Annaeus, l'Ancien 1932 Controve...","Seneca, Lucius Annaeus, l'Ancien Controverses...","[-0.0008525855, -0.05153799, -0.022173198, -0...."
1,1,1,https://play.google.com/store/books/details?id...,NaN,NaN,http://books.google.com/books/content?id=m5qZA...,fr,1932.0,Garnier,Philosophy,...,Controverses ; et Suasoires,264,Controverses ; et Suasoires,Controverses ; et Suasoires,"Seneca, Lucius Annaeus, l'Ancien Controverses ...","Seneca, Lucius Annaeus, l'Ancien 1932 Controve...","Seneca, Lucius Annaeus, l'Ancien 1932 Controve...","Seneca, Lucius Annaeus, l'Ancien 1932 Controve...","Seneca, Lucius Annaeus, l'Ancien Controverses...","[-0.0008525855, -0.05153799, -0.022173198, -0...."
2,2,2,https://books.google.com/books/about/Oeuvres_P...,NaN,2070108295,NaN,fr,1990.0,puis Gallimard; La Pléiade,NaN,...,"Oeuvres / Poésies de jeunesse, poésies diverse...",269,"Oeuvres / Poésies de jeunesse, poésies diverse...","Oeuvres / Poésies de jeunesse, poésies diverse...","Baudelaire, Charles Oeuvres / Poésies de jeune...","Baudelaire, Charles 1990 Oeuvres / Poésies de ...","Baudelaire, Charles 1990 Oeuvres / Poésies de ...","Baudelaire, Charles 1990 Oeuvres / Poésies de ...","Baudelaire, Charles Oeuvres / Poésies de jeun...","[0.029804604, -0.051942147, -0.016050369, -0.0..."
3,3,3,https://books.google.com/books/about/L_halluci...,L'Hallucination artistique (la formule est de ...,2952930295,NaN,fr,2012.0,F Alcan,Art and literature,...,L'hallucination,367,L'hallucination,L'hallucination L'Hallucination artistique (la...,"Quercy, Pierre L'hallucination L'Hallucination...","Quercy, Pierre 2012 L'hallucination L'Hallucin...","Quercy, Pierre 2012 L'hallucination","Quercy, Pierre 2012 L'hallucination Art and li...","Quercy, Pierre L'hallucination Art and litera...","[0.027692987, -0.02927246, -0.055086266, -0.01..."
4,4,4,https://books.google.com/books/about/Les_droit...,NaN,NaN,http://books.google.com/books/content?id=yo7dw...,fr,NaN,M Giard et E Brière,droits réels--* droit civil--Suisse; Sachenrec...,...,Les droits réels dans le Code civil suisse,393,Les droits réels dans le Code civil suisse,Les droits réels dans le Code civil suisse,"Wieland, Karl, Les droits réels dans le Code c...","Wieland, Karl, Les droits réels dans le Code ...","Wieland, Karl, Les droits réels dans le Code ...","Wieland, Karl, Les droits réels dans le Code ...","Wieland, Karl, Les droits réels dans le Code ...","[0.0069762063, -0.028140968, -0.034576803, 0.0..."


,user_id,book_id,timestamp
0,4456,8581,1.687541e+09
1,142,1964,1.679585e+09
2,362,3705,1.706872e+09
3,1809,11317,1.673533e+09
4,4384,1323,1.681402e+09


,Unnamed: 0.1,Unnamed: 0,CanonicalLink,Description,ISBN,ImageLink,Language,PublishedDate,Publisher,Subjects,...,google_api_title,book_id,title_clean,title_description,author_title_description,author_date_title_description,author_date_title,author_date_title_subjects,author_title_subjects,embedding
0,0,0,https://play.google.com/store/books/details?id...,NaN,NaN,http://books.google.com/books/content?id=m5qZA...,fr,1932.0,Garnier frères,Philosophy,...,Controverses ; et Suasoires,261,Controverses ; et Suasoires,Controverses ; et Suasoires,"Seneca, Lucius Annaeus, l'Ancien Controverses ...","Seneca, Lucius Annaeus, l'Ancien 1932 Controve...","Seneca, Lucius Annaeus, l'Ancien 1932 Controve...","Seneca, Lucius Annaeus, l'Ancien 1932 Controve...","Seneca, Lucius Annaeus, l'Ancien Controverses...","[-0.0008525855, -0.05153799, -0.022173198, -0...."
1,1,1,https://play.google.com/store/books/details?id...,NaN,NaN,http://books.google.com/books/content?id=m5qZA...,fr,1932.0,Garnier,Philosophy,...,Controverses ; et Suasoires,264,Controverses ; et Suasoires,Controverses ; et Suasoires,"Seneca, Lucius Annaeus, l'Ancien Controverses ...","Seneca, Lucius Annaeus, l'Ancien 1932 Controve...","Seneca, Lucius Annaeus, l'Ancien 1932 Controve...","Seneca, Lucius Annaeus, l'Ancien 1932 Controve...","Seneca, Lucius Annaeus, l'Ancien Controverses...","[-0.0008525855, -0.05153799, -0.022173198, -0...."
2,2,2,https://books.google.com/books/about/Oeuvres_P...,NaN,2070108295,NaN,fr,1990.0,puis Gallimard; La Pléiade,NaN,...,"Oeuvres / Poésies de jeunesse, poésies diverse...",269,"Oeuvres / Poésies de jeunesse, poésies diverse...","Oeuvres / Poésies de jeunesse, poésies diverse...","Baudelaire, Charles Oeuvres / Poésies de jeune...","Baudelaire, Charles 1990 Oeuvres / Poésies de ...","Baudelaire, Charles 1990 Oeuvres / Poésies de ...","Baudelaire, Charles 1990 Oeuvres / Poésies de ...","Baudelaire, Charles Oeuvres / Poésies de jeun...","[0.029804604, -0.051942147, -0.016050369, -0.0..."
3,3,3,https://books.google.com/books/about/L_halluci...,L'Hallucination artistique (la formule est de ...,2952930295,NaN,fr,2012.0,F Alcan,Art and literature,...,L'hallucination,367,L'hallucination,L'hallucination L'Hallucination artistique (la...,"Quercy, Pierre L'hallucination L'Hallucination...","Quercy, Pierre 2012 L'hallucination L'Hallucin...","Quercy, Pierre 2012 L'hallucination","Quercy, Pierre 2012 L'hallucination Art and li...","Quercy, Pierre L'hallucination Art and litera...","[0.027692987, -0.02927246, -0.055086266, -0.01..."
4,4,4,https://books.google.com/books/about/Les_droit...,NaN,NaN,http://books.google.com/books/content?id=yo7dw...,fr,NaN,M Giard et E Brière,droits réels--* droit civil--Suisse; Sachenrec...,...,Les droits réels dans le Code civil suisse,393,Les droits réels dans le Code civil suisse,Les droits réels dans le Code civil suisse,"Wieland, Karl, Les droits réels dans le Code c...","Wieland, Karl, Les droits réels dans le Code ...","Wieland, Karl, Les droits réels dans le Code ...","Wieland, Karl, Les droits réels dans le Code ...","Wieland, Karl, Les droits réels dans le Code ...","[0.0069762063, -0.028140968, -0.034576803, 0.0..."


In [23]:
n_users = interactions.user_id.nunique()
n_items = items.book_id.nunique()
print(f'Number of users = {n_users}, \n Number of books = {n_items} \n Number of interactions = {len(interactions)}')

Number of users = 7838, 
 Number of books = 15291 
 Number of interactions = 87047


####TRAIN/TEST SPLIT

In [24]:
interactions = interactions.sort_values(["user_id", "timestamp"])
interactions.head(10)
interactions["pct_rank"] = interactions.groupby("user_id")["timestamp"].rank(pct=True, method='dense')
interactions.reset_index(inplace=True, drop=True)
interactions.head(10)
train_data = interactions[interactions["pct_rank"] < 0.8]
test_data = interactions[interactions["pct_rank"] >= 0.8]
print("Training set size:", train_data.shape[0])
print("Testing set size:", test_data.shape[0])

Training set size: 65419
Testing set size: 21628


In [25]:
# Define a function to create the data matrix
def create_data_matrix(data, n_users, n_items):
    """
    This function returns a numpy matrix with shape (n_users, n_items).
    Each entry is a binary value indicating positive interaction.
    """
    data_matrix = np.zeros((n_users, n_items))
    data_matrix[data["user_id"].values, data["book_id"].values] = 1
    return data_matrix

In [26]:
# Create the training and testing matrices
train_data_matrix = create_data_matrix(train_data, n_users, n_items)
test_data_matrix = create_data_matrix(test_data, n_users, n_items)

# Display the matrices to understand their structure
print('train_data_matrix')
print(train_data_matrix)
print("number of non-zero values: ", np.sum(train_data_matrix))
print('test_data_matrix')
print(test_data_matrix)
print("number of non-zero values: ", np.sum(test_data_matrix))

train_data_matrix
[[1. 1. 1. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
number of non-zero values:  49689.0
test_data_matrix
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
number of non-zero values:  19409.0


#### BASIC DEFINITIONS NEEDED

In [27]:
# Define a function to create the data matrix
def create_data_matrix(data, n_users, n_items):
    """
    This function returns a numpy matrix with shape (n_users, n_items).
    Each entry is a binary value indicating positive interaction.
    """
    data_matrix = np.zeros((n_users, n_items))
    data_matrix[data["user_id"].values, data["book_id"].values] = 1
    return data_matrix

In [28]:
# Define the function to predict interactions based on item similarity
def item_based_predict(interactions, similarity, epsilon=1e-9):
    """
    Predicts user-item interactions based on item-item similarity.
    Parameters:
        interactions (numpy array): The user-item interaction matrix.
        similarity (numpy array): The item-item similarity matrix.
        epsilon (float): Small constant added to the denominator to avoid division by zero.
    Returns:
        numpy array: The predicted interaction scores for each user-item pair.
    """
    # np.dot does the matrix multiplication. Here we are calculating the
    # weighted sum of interactions based on item similarity
    pred = similarity.dot(interactions.T) / (similarity.sum(axis=1)[:, np.newaxis] + epsilon)
    return pred.T  # Transpose to get users as rows and items as columns

In [29]:
# Recommendation frame generation
def create_recommendation_table(user_predictions, top_n=10, separator=" "):
    """
    Creates a table of top-N recommendations for each user.

    Args:
        user_predictions (numpy.ndarray): Rows = users, columns = items. Predicted scores.
        top_n (int): Number of top recommendations per user.
        separator (str): Delimiter to join recommended book IDs.

    Returns:
        pandas.DataFrame: Columns = ['user_id', 'recommendation'].
    """
    recommendations = []
    num_users = user_predictions.shape[0]

    for user_id in range(num_users):
        top_items = np.argsort(user_predictions[user_id, :])[-top_n:][::-1]
        recommendations.append({
            'user_id': user_id,
            'recommendation': separator.join(map(str, top_items))
        })

    return pd.DataFrame(recommendations)

In [30]:
# Implement the precision_recall_at_k function
def precision_recall_at_k(prediction, ground_truth, k=10):
    """
    Calculates Precision@K and Recall@K for top-K recommendations.
    Parameters:
        prediction (numpy array): The predicted interaction matrix with scores.
        ground_truth (numpy array): The ground truth interaction matrix (binary).
        k (int): Number of top recommendations to consider.
    Returns:
        precision_at_k (float): The average precision@K over all users.
        recall_at_k (float): The average recall@K over all users.
    """
    num_users = prediction.shape[0]
    precision_at_k, recall_at_k = 0, 0

    for user in range(num_users):
        # TODO: Get the indices of the top-K items for the user based on predicted scores
        top_k_items = np.argsort(prediction[user, :])[-k:]

        # TODO: Calculate the number of relevant items in the top-K items for the user
        relevant_items_in_top_k = np.isin(top_k_items, np.where(ground_truth[user, :] == 1)[0]).sum()

        # TODO: Calculate the total number of relevant items for the user
        total_relevant_items = ground_truth[user, :].sum()

        # Precision@K and Recall@K for this user
        precision_at_k += relevant_items_in_top_k / k
        recall_at_k += relevant_items_in_top_k / total_relevant_items if total_relevant_items > 0 else 0

    # Average Precision@K and Recall@K over all users
    precision_at_k /= num_users
    recall_at_k /= num_users

    return precision_at_k, recall_at_k

#### API Embeddings tests
Try the different embeddings - what gives best result? <BR>
**author_date_title_DESCRIPTION:** <BR>
Item-based EMBED Precision@K: 0.03849196223526418  <BR>
Item-based EMBED Recall@K: 0.21465701620473912<BR>

**author_date_title_SUBJECT:**
Item-based EMBED Precision@K: 0.038645062515948056 <BR>
Item-based EMBED Recall@K: 0.21499394666978877

**title_description:**<BR>
Item-based EMBED Precision@K: 0.03685889257463616 <BR>
Item-based EMBED Recall@K: 0.20913895560787144


In [39]:
# Choose the single embedding column that already holds the full vector
embedding_col = 'embedding'  # replace with actual column name if different

# Parse the embedding strings into numpy arrays
items[embedding_col] = items[embedding_col].apply(lambda x: np.fromstring(x.strip('[]'), sep=','))

# Drop rows with missing or malformed embeddings
valid_items = items[items[embedding_col].notna()].reset_index(drop=True)

# Stack all embeddings into a matrix
embedding_matrix = np.vstack(valid_items[embedding_col].values)

# Compute item-item cosine similarity
embedding_sim = cosine_similarity(embedding_matrix)

# Optional: normalize the similarity matrix row-wise
embedding_sim = normalize(embedding_sim)

In [40]:
# Create the training and testing matrices
train_data_matrix = create_data_matrix(train_data, n_users, n_items)
test_data_matrix = create_data_matrix(test_data, n_users, n_items)

# Display the matrices to understand their structure
print('train_data_matrix')
print(train_data_matrix)
print("number of non-zero values: ", np.sum(train_data_matrix))
print('test_data_matrix')
print(test_data_matrix)
print("number of non-zero values: ", np.sum(test_data_matrix))

train_data_matrix
[[1. 1. 1. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
number of non-zero values:  49689.0
test_data_matrix
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
number of non-zero values:  19409.0


In [41]:
# Calculate the item-based predictions for positive interactions
item_EMBED_prediction = item_based_predict(train_data_matrix, embedding_sim)
print("Predicted Interaction Matrix:")
print(item_EMBED_prediction)
print(item_EMBED_prediction.shape)

Predicted Interaction Matrix:
[[0.00130855 0.00130855 0.00129573 ... 0.00122333 0.00123584 0.0012079 ]
 [0.00059032 0.00059032 0.00059832 ... 0.00059292 0.00058583 0.00059743]
 [0.00277832 0.00277832 0.00284124 ... 0.00276736 0.00280048 0.00269765]
 ...
 [0.00012931 0.00012931 0.00012242 ... 0.00013128 0.00013429 0.00013116]
 [0.00012529 0.00012529 0.00012933 ... 0.00012827 0.00012684 0.00012874]
 [0.00013354 0.00013354 0.00013832 ... 0.00012578 0.0001205  0.00012727]]
(7838, 15291)


In [43]:
# CHECK PRECISION & RECALL NOT YET WITH CROSS-VALIDATION [OVERFITTING PROBLEM THOUGH]
precision_item_k, recall_item_k = precision_recall_at_k(item_EMBED_prediction, test_data_matrix, k=10)
print('Item-based EMBED Precision@K:', precision_item_k)
print('Item-based EMBED Recall@K:', recall_item_k)

Item-based EMBED Precision@K: 0.03685889257463616
Item-based EMBED Recall@K: 0.20913895560787144
